# Parameter transformations and jacobians

We use the `sympy` package to compute parameter transformations and jacobians which are relevant for parameter estimation.

In [ ]:
import sympy as sp
from IPython.display import display
sp.init_printing()

## Describing masses

The masses of a compact binary system are described by two parameters. The most suitable parametrization depends on the problem at hand. For modelling astrophysically-informed prior probabilities on the merger rate density per unit mass, it is commont to use $(m_1, q)$, where $m_1$ is the primary mass (i.e,  the mass of the heavier object) and $0 \leq q \leq 1$ is the ratio between the objects' masses.

The mass parameters that appear in analytical approximations are, instead, the symmetric mass ratio,

$$ \nu = \frac{m_1 m_2}{(m_1 + m_2)^2},$$

which is bounded between $(0, 1/4]$, and the chirp mass

$$ M_c = (m_1 + m_2) \nu^{3/5}. $$

In this section, we describe a couple of simple transformations between the set of parameters $\{m_1, m_2, M_t, q, M_c, \nu \}$ and compose them sequentially in order to obtain transformations between arbitrary pairs of parameters. We implement this logic in the `get_transform_and_jacobian` method. In a nutshell, it updates each mass parameter with the composition of expressions defined in `subs`.

### Component masses to other parametrizations

In [ ]:
def pretty_print(outputs, expressions, jacobian, determinant):
  eqs = [sp.Eq(o, sp.simplify(e), evaluate=False) for o, e in zip(outputs, expressions)]
  print("Expressions, jacobian, determinant:")
  display((eqs, sp.simplify(jacobian), sp.simplify(determinant)))

def get_transform_and_jacobian(inputs, outputs, subs):
    out_exprs = sp.Matrix(outputs).subs(subs[::-1])
    jacobian = out_exprs.jacobian(inputs)
    return out_exprs, jacobian

m_1, m_2, M_t, M_c, q, nu = sp.symbols("m_1 m_2 M_t M_c q nu")

subs = [
  (M_t, m_1 + m_2),
  (q, m_2 / m_1),
  (nu, q / (1 + q) ** 2),
  (M_c, M_t * nu ** sp.Rational(3, 5))
]
inputs = (m_1, m_2)
for outputs in [(m_1, q), (M_t, q), (M_c, nu)]:
  expressions, jacobian = get_transform_and_jacobian(inputs, outputs, subs)
  det = jacobian.det()
  pretty_print(outputs, expressions, jacobian, det)

### Primary mass and mass ratio to other parametrizations

In [ ]:
subs = [
  (m_2, m_1 * q),
  (M_t, m_1 + m_2),
  (nu, q / (1 + q) ** 2),
  (M_c, M_t * nu ** sp.Rational(3, 5))
]
inputs = (m_1, q)
for outputs in [(m_1, m_2), (M_t, q), (M_c, nu)]:
  expressions, jacobian = get_transform_and_jacobian(inputs, outputs, subs)
  det = jacobian.det()
  pretty_print(outputs, expressions, jacobian, det)

### Total mass and mass ratio to other parametrizations

In [ ]:
subs = [
  (m_1, M_t / (1 + q)),
  (m_2, M_t - m_1),
  (nu, q / (1 + q) ** 2),
  (M_c, M_t * nu ** sp.Rational(3, 5))
]
inputs = (M_t, q)
for outputs in [(m_1, m_2), (m_1, q), (M_c, nu)]:
  expressions, jacobian = get_transform_and_jacobian(inputs, outputs, subs)
  det = jacobian.det()
  pretty_print(outputs, expressions, jacobian, det)

### Chirp mass and symmetric mass ratio to other parametrizations

In [ ]:
subs = [
  (M_t, M_c * nu ** sp.Rational(-3, 5)),
  (q, -1 + sp.Rational(1, 2) / nu * (1 + sp.sqrt(1 - 4 * nu))),
  (m_1, M_t / (1 + q)),
  (m_2, M_t - m_1)
]
inputs = (M_c, nu)
for outputs in [(m_1, m_2), (m_1, q), (M_t, q)]:
  expressions, jacobian = get_transform_and_jacobian(inputs, outputs, subs)
  det = jacobian.det()
  pretty_print(outputs, expressions, jacobian, det)